# Get Jobs

[AutomationAPI Documentation](https://documents.bmc.com/supportu/API/Monthly/en-US/Documentation/API_Services_DeployService.htm#deploy3)


The get_jobs() method in the Control-M Python Client provides an easy way to retrieve job and folder definitions from the Control-M Server. It wraps the deploy jobs::get AAPI command and deserializes the resulting JSON into Python objects, allowing users to fetch and work with jobs and folders in a way that mirrors the state before deployment.

This guide demonstrates how to use get_jobs() to fetch jobs from the Control-M server.

In [2]:
from ctm_python_client.core.workflow import Workflow, WorkflowDefaults
from ctm_python_client.core.comm import Environment
from aapi import *

# Step 1: Define the environment (Control-M Workbench in this case)
env = Environment.create_workbench('workbench')

# Step 2: Define your workflow with jobs and folders
workflow = Workflow(env, WorkflowDefaults(run_as='workbench'))
workflow.clear_all()
run_as_dummy = True

# Define jobs
databricksLoad = JobDatabricks(
    'LoadForecasts',
    connection_profile='DATABRICKS',
    databricks_job_id='991955986358417',
    parameters='"params":{}',
    idempotency_token='tokeni_%%ORDERID',
    run_as_dummy=run_as_dummy
)

sagemaker_job = JobAwsSageMaker(
    'InventoryForecastModel',
    connection_profile='SAGEMAKER',
    pipeline_name='InferencePipeline',
    idempotency_token='Token_ControlM_for_SageMaker%%ORDERID',
    add_parameters='checked',
    parameters='{"Name":"input_file", "Value": "file"}',
    run_as_dummy=run_as_dummy,
    retry_pipeline_execution='unchecked',
    rerun_limit=Job.RerunLimit(times='5')
)

# Define folders
folder1 = Folder('Folder_Sanity_1', controlm_server='workbench', job_list=[databricksLoad])
folder2 = Folder('Folder_Sanity_2', controlm_server='workbench', job_list=[sagemaker_job])

workflow.add(folder1)
workflow.add(folder2)

# Step 3: Deploy workflow
workflow.build()
workflow.deploy()

# Step 4: Retrieve jobs after deployment
workflow_actual = Workflow.get_jobs(env, server="workbench", folder="Folder_Sanity_*")

print(workflow.dumps_json(indent=2))


{
  "Folder_Sanity_1": {
    "Type": "Folder",
    "ControlmServer": "workbench",
    "LoadForecasts": {
      "Type": "Job:Databricks",
      "RunAs": "workbench",
      "RunAsDummy": true,
      "ConnectionProfile": "DATABRICKS",
      "Databricks Job ID": "991955986358417",
      "Parameters": "\"params\":{}",
      "Idempotency Token": "tokeni_%%ORDERID"
    },
    "RunAs": "workbench"
  },
  "Folder_Sanity_2": {
    "Type": "Folder",
    "ControlmServer": "workbench",
    "InventoryForecastModel": {
      "Type": "Job:AWS SageMaker",
      "RerunLimit": {
        "Times": "5"
      },
      "RunAs": "workbench",
      "RunAsDummy": true,
      "ConnectionProfile": "SAGEMAKER",
      "Pipeline Name": "InferencePipeline",
      "Idempotency Token": "Token_ControlM_for_SageMaker%%ORDERID",
      "Add Parameters": "checked",
      "Parameters": "{\"Name\":\"input_file\", \"Value\": \"file\"}",
      "Retry Pipeline Execution": "unchecked"
    },
    "RunAs": "workbench"
  }
}


## Arguments

- **environment: Environment** :
  The Control-M environment to connect to. Required.  
  This object defines the Control-M endpoint (Automation API endpoint, same as Control-M/EM), the authentication method, and the environment mode (on-prem or SaaS).  
  You can create an Environment instance using one of the following static methods:  
  - `create_workbench()` – for local development with Workbench.  
  - `create_onprem(host, username, password)` – for on-premises Control-M using username/password authentication.  
  - `create_saas(endpoint, api_key)` – for Helix Control-M (SaaS) using an API key.  
  The environment determines how to authenticate and which API variant to use, depending on whether the backend is Control-M or Helix Control-M.


- **server: str** :
The exact Control-M/Server name to query. Required.  
No wildcards allowed.

- **folder: str** :
The folder name or pattern to fetch. Required.  
Supports wildcards (e.g., "MyFolder_*"). Filters jobs by folder name.

- **job: str** (Not supported yet)  
  Currently not supported. The API ignores this parameter if provided.